In [44]:
import pandas as pd

In [45]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [47]:
df = df.sort_values("date")

In [48]:
df = df.reset_index(drop=True)

In [49]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [50]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

df = df.groupby("team", group_keys=False).apply(add_target).reset_index(drop=True)
df = df.copy()


/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/3664484873.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/3664484873.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/3664484873.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

In [51]:
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/2793819745.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/27938197

In [52]:
df["won"].value_counts()

won
False    8886
True     8886
Name: count, dtype: int64

In [53]:
df["target"].value_counts()

target
1    8872
0    8870
2      30
Name: count, dtype: int64

In [54]:
nulls = pd.isnull(df).sum()

In [55]:
nulls = nulls[nulls > 0]

In [56]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [57]:
valid_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=142)

In [58]:
df = df[valid_columns].copy()

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline

logistic_model = LogisticRegression(max_iter=1000)

split = TimeSeriesSplit(n_splits=3)

selector = SelectKBest(score_func=f_classif, k=30)

search_space = {
    "classification__penalty": ["l1", "l2"],
    "classification__C": [0.1, 1, 10],
    "classification__solver": ["liblinear", "lbfgs"]
}

pipeline = Pipeline(steps=[('feature_selection', selector), ('classification', logistic_model)])

gs = GridSearchCV(estimator=pipeline, param_grid=search_space, cv=split, scoring='accuracy')

In [61]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [62]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [64]:
gs.fit(df[selected_columns], df["target"])

/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 98] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 98] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


[LibLinear]iter   1  #CD cycles 1
iter   2  #CD cycles 4
iter   3  #CD cycles 1
iter   4  #CD cycles 5
iter   5  #CD cycles 4
iter   6  #CD cycles 1
iter   7  #CD cycles 36
iter   8  #CD cycles 1
iter   9  #CD cycles 63
iter  10  #CD cycles 1
iter  11  #CD cycles 9
iter  12  #CD cycles 1
iter  13  #CD cycles 23
iter  14  #CD cycles 2
iter  15  #CD cycles 1
iter  16  #CD cycles 30
iter  17  #CD cycles 2
optimization finished, #iter = 17
Objective value = 307.630308
#nonzeros/#features = 9/31
[LibLinear]iter   1  #CD cycles 1
iter   2  #CD cycles 4
iter   3  #CD cycles 1
iter   4  #CD cycles 10
iter   5  #CD cycles 4
iter   6  #CD cycles 1
iter   7  #CD cycles 12
iter   8  #CD cycles 2
iter   9  #CD cycles 6
iter  10  #CD cycles 1
iter  11  #CD cycles 70
iter  12  #CD cycles 1
iter  13  #CD cycles 203
iter  14  #CD cycles 3
iter  15  #CD cycles 10
iter  16  #CD cycles 2
iter  17  #CD cycles 6
iter  18  #CD cycles 1
iter  19  #CD cycles 189
iter  20  #CD cycles 4
iter  21  #CD cycles 1
it

/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 98] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


[LibLinear]iter   1  #CD cycles 1
iter   2  #CD cycles 3
iter   3  #CD cycles 1
iter   4  #CD cycles 11
iter   5  #CD cycles 1
iter   6  #CD cycles 17
iter   7  #CD cycles 5
iter   8  #CD cycles 1
iter   9  #CD cycles 112
iter  10  #CD cycles 1
iter  11  #CD cycles 137
iter  12  #CD cycles 1
iter  13  #CD cycles 117
iter  14  #CD cycles 2
iter  15  #CD cycles 1
iter  16  #CD cycles 127
iter  17  #CD cycles 1
optimization finished, #iter = 17
Objective value = 919.792195
#nonzeros/#features = 11/31
[LibLinear]iter  1 act 2.776e+00 pre 2.765e+00 delta 1.044e+00 f 3.080e+02 |g| 7.713e+00 CG   4
iter  2 act 5.139e-02 pre 5.139e-02 delta 1.044e+00 f 3.052e+02 |g| 4.546e-01 CG   6
iter  3 act 6.247e-05 pre 6.247e-05 delta 1.044e+00 f 3.051e+02 |g| 1.525e-02 CG   6
iter  4 act 1.628e-07 pre 1.628e-07 delta 1.044e+00 f 3.051e+02 |g| 1.009e-03 CG   6
[LibLinear]iter  1 act 6.123e+00 pre 6.088e+00 delta 1.065e+00 f 6.159e+02 |g| 1.636e+01 CG   4
iter  2 act 2.133e-01 pre 2.133e-01 delta 1.065e+0

/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 98] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 98] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.

...

/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 98] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
 This problem is unconstrained.
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 98] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  4.43044D-03

At iterate   50    f=  6.86390D-01    |proj g|=  4.80847D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     78     89      1     0     0   7.741D-05   6.863D-01
  F =  0.68630853745345832     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           93     M =           10

At X0         0 v

/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
9 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 


At iterate   50    f=  6.96372D-01    |proj g|=  3.82690D-04

At iterate  100    f=  6.96020D-01    |proj g|=  1.89421D-04

At iterate  150    f=  6.95832D-01    |proj g|=  1.45295D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   93    194    225      1     0     0   9.439D-05   6.957D-01
  F =  0.69572657173792030     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=Pipeline(steps=[('feature_selection', SelectKBest(k=30)),
                                       ('classification',
                                        LogisticRegression(max_iter=1000,
                                                           verbose=True))]),
             param_grid={'classification__C': [0.1, 1, 10],
                         'classification__penalty': ['l1', 'l2'],
                         'classification__solver': ['liblinear', 'lbfgs']},
             scoring='accuracy')

In [65]:
# Najlepszy model po GridSearchCV
best_model = gs.best_estimator_

# Wyświetlanie najlepszych parametrów
print("Najlepsze parametry: ", gs.best_params_)

best_rfe = best_model.named_steps['feature_selection']

Najlepsze parametry:  {'classification__C': 10, 'classification__penalty': 'l2', 'classification__solver': 'lbfgs'}


In [66]:
predictors = list(selected_columns[best_rfe.get_support()])

In [67]:
predictors

['fg%',
 'pts',
 'ts%',
 'efg%',
 'trb%',
 'blk%',
 'ortg',
 'drtg',
 'fg_max',
 'fga_max',
 'ast_max',
 'pts_max',
 '+/-_max',
 'drtg_max',
 'total',
 'fg_opp',
 'fg%_opp',
 'drb_opp',
 'trb_opp',
 'blk_opp',
 'pts_opp',
 'ts%_opp',
 'efg%_opp',
 'trb%_opp',
 'blk%_opp',
 'ortg_opp',
 'drtg_opp',
 '+/-_max_opp',
 'drtg_max_opp',
 'total_opp']

In [68]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [69]:
predictions = backtest(df, best_model, predictors)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  4.36063D-03

At iterate   50    f=  6.84921D-01    |proj g|=  6.42285D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     74     88      1     0     0   9.280D-05   6.848D-01
  F =  0.68480979562778443     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 v

 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  4.95078D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     45     55      1     0     0   9.875D-05   6.852D-01
  F =  0.68516179684470080     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.9

 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  4.58284D-03

At iterate   50    f=  6.85991D-01    |proj g|=  1.39333D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     56     65      1     0     0   9.269D-05   6.860D-01
  F =  0.68597780073526371     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


In [70]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

0.5396901453441942

In [72]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

def find_team_averages(team):
    numeric_cols = team.select_dtypes(include="number")
    rolling = numeric_cols.rolling(10).mean()
    
    rolling[["team", "season"]] = team[["team", "season"]]
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/410169298.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [73]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,season,team
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,NOP
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,CLE
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,CHI
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,GSW
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.381818,0.292647,0.428230,0.468966,0.477273,0.448100,0.434884,0.373016,0.764177,...,0.0570,0.1113,0.471908,0.170603,0.431754,0.522353,0.348077,0.5,2022,BOS
17768,0.0,0.502273,0.364706,0.517703,0.455172,0.481818,0.440736,0.320930,0.282540,0.757993,...,0.0716,0.1171,0.374109,0.321566,0.642654,0.564706,0.392308,0.4,2022,GSW
17769,0.0,0.354545,0.279412,0.404545,0.437931,0.465152,0.429572,0.434884,0.385714,0.736639,...,0.0591,0.1113,0.483229,0.174711,0.438863,0.483529,0.350000,0.5,2022,BOS
17770,0.0,0.354545,0.294118,0.389952,0.434483,0.459091,0.431710,0.406977,0.357143,0.754142,...,0.0572,0.1111,0.483229,0.172144,0.460190,0.472941,0.344231,0.5,2022,BOS


In [74]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [75]:
df = df.dropna()

In [76]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_7386/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x,

In [77]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

In [78]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,SAC,TOR,TOR,SAC,2015-11-15
1,TOR,SAC,SAC,TOR,2015-11-15
2,CLE,DET,DET,CLE,2015-11-17
3,GSW,TOR,TOR,GSW,2015-11-17
4,DEN,NOP,NOP,DEN,2015-11-17
...,...,...,...,...,...
15769,BOS,GSW,GSW,BOS,2022-06-10
15770,GSW,BOS,BOS,GSW,2022-06-13
15771,BOS,GSW,GSW,BOS,2022-06-13
15772,GSW,BOS,BOS,GSW,2022-06-16


In [79]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [80]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_10_x',
 'team_opp_next_x',
 'date_next',
 'team_10_y',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [81]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
selector.fit(full[selected_columns], full["target"])

/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 30  98 166 234 304 372] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/karol/Documents/studia/licencjat/nba ml/venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


SelectKBest(k=30)

In [82]:
predictors = list(selected_columns[selector.get_support()])

In [83]:
predictors

['fg%_10_x',
 'pts_10_x',
 'ts%_10_x',
 'efg%_10_x',
 'trb%_10_x',
 'ortg_10_x',
 'drtg_10_x',
 'pts_max_10_x',
 '+/-_max_10_x',
 'total_10_x',
 'blk_opp_10_x',
 'trb%_opp_10_x',
 'ortg_opp_10_x',
 'drtg_opp_10_x',
 '+/-_max_opp_10_x',
 'drtg_max_opp_10_x',
 'home_next',
 'fg%_10_y',
 'pts_10_y',
 'ts%_10_y',
 'efg%_10_y',
 'ortg_10_y',
 'pts_max_10_y',
 '+/-_max_10_y',
 'total_10_y',
 'blk_opp_10_y',
 'ortg_opp_10_y',
 'drtg_opp_10_y',
 '+/-_max_opp_10_y',
 'drtg_max_opp_10_y']

In [84]:
predictions = backtest(full, logistic_model, predictors)

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  4.54351D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     39     43      1     0     0   8.011D-05   6.287D-01
  F =  0.62870766955025204     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.9

 This problem is unconstrained.
 This problem is unconstrained.


In [85]:
wynik = accuracy_score(predictions["actual"], predictions["prediction"])

In [86]:
print(f"wynik to {wynik*100:.4f}"+"% dokładności osiągnięte przez najlepszy model Logistycznej Regresji z selectorem Select K Best")

wynik to 63.4316% dokładności osiągnięte przez najlepszy model Logistycznej   Select K Best
